In [1]:
import sys
import os
# import calib_eq_odds as eq
cwd = os.getcwd()
print(cwd)
sys.path.append(cwd + "/Post-Processing")
from calib_eq_odds import Model
import pandas as pd
import numpy as np
import torch
import torch.nn as nn


fn_rate = 1
fp_rate = 0

kiva_label = "dataAug/kiva_label.csv"
student_label = "dataAug/student_label.csv" 

/home/li3975/cs587


In [10]:
data = pd.read_csv(kiva_label)
data['repayment_interval'] = data['repayment_interval'].replace(2, 1)
train_data = data [0:20000]
target = train_data['repayment_interval']
target = torch.tensor(target.values).float()
X = train_data.drop(['repayment_interval'], axis=1)
X = torch.tensor(X.values).float()
gender_index = data.columns.get_loc('borrower_genders')
val_data = data[20000: 40000]
val_target = val_data['repayment_interval']
val_target = torch.tensor(val_target.values).float()
val_X = val_data.drop(['repayment_interval'], axis=1)
val_X = torch.tensor(val_X.values).float()
test_data = data[40000:]
test_target = test_data['repayment_interval']
test_target = torch.tensor(test_target.values).float()
test_X = test_data.drop(['repayment_interval'], axis=1)
test_X = torch.tensor(test_X.values).float()


In [11]:
class LogisticRegression(nn.Module):
    def __init__(self):
        # Call the parent constructor to initialize the nn.Module
        super(LogisticRegression, self).__init__()

        # Define a linear transformation layer (weights and bias) for the model.
        # `x_female.shape[1]` specifies the number of input features, and `out_features=1` indicates a single output for binary classification.
        # The bias term is included by setting bias=True, allowing the model to learn an intercept.
        self.w = nn.Linear(9, out_features=1, bias=True)

        # Initialize a sigmoid activation function, which will be used to convert logits to probabilities.
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Define the forward pass for the model.
        # Apply the linear transformation to the input `x`.
        w = self.w(x)

        # Apply the sigmoid function to the output of the linear layer.
        # This converts the logits (raw output of the linear layer) to probabilities in the range (0, 1).
        output = self.sigmoid(w)

        # Return the output probabilities.
        return output


In [18]:
def train():
    model = LogisticRegression()
    ceriation = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(X)
        loss = ceriation(output.squeeze(1), target)
        loss.backward()
        optimizer.step()
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, 100, loss.item()))
    return model

In [ ]:

model = train()

epoch [1/100], loss:90.7762
epoch [2/100], loss:88.7517
epoch [3/100], loss:87.3981
epoch [4/100], loss:86.0003
epoch [5/100], loss:84.6232
epoch [6/100], loss:82.8495
epoch [7/100], loss:80.6607
epoch [8/100], loss:77.6440
epoch [9/100], loss:74.4764
epoch [10/100], loss:71.1278
epoch [11/100], loss:67.0021
epoch [12/100], loss:62.3932
epoch [13/100], loss:56.8868
epoch [14/100], loss:50.8296
epoch [15/100], loss:43.3504
epoch [16/100], loss:34.1392
epoch [17/100], loss:24.0827
epoch [18/100], loss:13.4962
epoch [19/100], loss:4.7600
epoch [20/100], loss:4.5659
epoch [21/100], loss:5.3234
epoch [22/100], loss:5.5040
epoch [23/100], loss:5.5213
epoch [24/100], loss:5.5257
epoch [25/100], loss:5.5258
epoch [26/100], loss:5.5300
epoch [27/100], loss:5.5300
epoch [28/100], loss:5.5300
epoch [29/100], loss:5.5300
epoch [30/100], loss:5.5300
epoch [31/100], loss:5.5300
epoch [32/100], loss:5.5300
epoch [33/100], loss:5.5300
epoch [34/100], loss:5.5300
epoch [35/100], loss:5.5300
epoch [36/1

In [23]:
# with no grad
group_0_val_pred = []
group_1_val_pred = []
group_0_val_gt= []
group_1_val_gt = []

group_0_test_pred = []
group_1_test_pred = []
group_0_test_gt= []
group_1_test_gt = []

def evaluated_nn(model, val_X, val_target, test_X, test_target):

    model.eval()
    with torch.no_grad():
        for x, y in zip(val_X, val_target):
            y_pred = model(x)
            if x[gender_index] == 0:
                group_0_val_pred.append(y_pred.item())
                group_0_val_gt.append(y.item())
            else:
                group_1_val_pred.append(y_pred.item())
                group_1_val_gt.append(y.item())
            # print(f'Prediction: {y_pred.item()}, True: {y.item()}')
        
        for x, y in zip(test_X, test_target):
            y_pred = model(x)
            if x[gender_index] == 0:
                group_0_test_pred.append(y_pred.item())
                group_0_test_gt.append(y.item())
            else:
                group_1_test_pred.append(y_pred.item())
                group_1_test_gt.append(y.item())
            # print(f'Prediction: {y_pred.item()}, True: {y.item()}')

evaluated_nn(model, val_X, val_target, test_X, test_target)

In [24]:
group_0_val_model = Model(np.array(group_0_val_pred), np.array(group_0_val_gt))
group_1_val_model = Model(np.array(group_1_val_pred), np.array(group_1_val_gt))
group_0_test_model = Model(np.array(group_0_test_pred), np.array(group_0_test_gt))
group_1_test_model = Model(np.array(group_1_test_pred), np.array(group_1_test_gt))

In [25]:
_, _, mix_rates = Model.calib_eq_odds(group_0_val_model, group_1_val_model, fp_rate, fn_rate)
calib_eq_odds_group_0_test_model, calib_eq_odds_group_1_test_model = Model.calib_eq_odds(
    group_0_test_model, group_1_test_model, fp_rate, fn_rate, mix_rates)
print('Original group 0 model:\n%s\n' % repr(group_0_test_model))
print('Original group 1 model:\n%s\n' % repr(group_1_test_model))
print('Equalized odds group 0 model:\n%s\n' % repr(calib_eq_odds_group_0_test_model))
print('Equalized odds group 1 model:\n%s\n' % repr(calib_eq_odds_group_1_test_model))

Original group 0 model:
Accuracy:	0.949
F.P. cost:	1.000
F.N. cost:	0.000
Base rate:	0.949
Avg. score:	1.000

Original group 1 model:
Accuracy:	0.780
F.P. cost:	1.000
F.N. cost:	0.000
Base rate:	0.780
Avg. score:	1.000

Equalized odds group 0 model:
Accuracy:	0.949
F.P. cost:	1.000
F.N. cost:	0.000
Base rate:	0.949
Avg. score:	1.000

Equalized odds group 1 model:
Accuracy:	0.780
F.P. cost:	1.000
F.N. cost:	0.000
Base rate:	0.780
Avg. score:	1.000



In [ ]:
data = pd.read_csv(student_label)
data['repayment_interval'] = data['repayment_interval'].replace(2, 1)
train_data = data [0:20000]
target = train_data['repayment_interval']
target = torch.tensor(target.values).float()
X = train_data.drop(['repayment_interval'], axis=1)
X = torch.tensor(X.values).float()
gender_index = data.columns.get_loc('Gender')
val_data = data[20000: 40000]
val_target = val_data['repayment_interval']
val_target = torch.tensor(val_target.values).float()
val_X = val_data.drop(['repayment_interval'], axis=1)
val_X = torch.tensor(val_X.values).float()
test_data = data[40000:]
test_target = test_data['repayment_interval']
test_target = torch.tensor(test_target.values).float()
test_X = test_data.drop(['repayment_interval'], axis=1)
test_X = torch.tensor(test_X.values).float()